In [1]:
from typing import Dict, Optional, List
import datetime as dt
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.contrib.latex_printer import latex_printer
import mpisppy.utils.sputils as sputils
from mpisppy.opt.ef import ExtensiveForm

[    0.00] Initializing mpi-sppy


In [2]:
def build_model(price: int, objective_limit: float = float('inf')) -> pyo.ConcreteModel:
    mdl = pyo.ConcreteModel()

    mdl.lot = pyo.Set(initialize=[1, 2])

    mdl.land_development_price = pyo.Param(mdl.lot, initialize={1: 600, 2: 100})
    mdl.build_now_price = pyo.Param(mdl.lot, initialize={1: 200, 2: 600})
    mdl.build_later_price = pyo.Param(mdl.lot, initialize={1: 220, 2: 660})

    mdl.develop_land = pyo.Var(mdl.lot, domain=pyo.Binary, name="Develop land")
    mdl.build_now = pyo.Var(mdl.lot, domain=pyo.Binary, name="Build now")
    mdl.build_later = pyo.Var(mdl.lot, domain=pyo.Binary, name="Build later")

    mdl.first_stage_costs = sum(
        mdl.land_development_price[lot] * mdl.develop_land[lot]
        + mdl.build_now_price[lot] * mdl.build_now[lot]
        for lot in mdl.lot
    )
    mdl.build_later_cost = sum(
        mdl.build_later_price[lot] * mdl.build_later[lot]
        for lot in mdl.lot
    )
    mdl.revenues = sum(
        price * (mdl.build_now[lot] + mdl.build_later[lot])
        for lot in mdl.lot
    )
    
    mdl.objective = pyo.Objective(
        expr=(
            mdl.revenues - mdl.first_stage_costs - mdl.build_later_cost
        ),
        sense=pyo.maximize,
    )

    mdl.constr_build_once = pyo.ConstraintList()
    for lot in mdl.lot:
        mdl.constr_build_once.add(mdl.build_now[lot] + mdl.build_later[lot] <= 1)

    mdl.constr_cost_of_land = pyo.ConstraintList()
    for lot in mdl.lot:
        mdl.constr_cost_of_land.add(mdl.develop_land[lot] >= (mdl.build_now[lot] + mdl.build_later[lot]))

    mdl.limit_optimal_solution = pyo.Constraint(
        expr=mdl.objective <= objective_limit
    )

    return mdl

In [7]:
scenarios = {
    "s0": 210,
    "s1": 1250,
    # "s2": 2500
}

def scenario_creator(scenario):
    price = scenarios[scenario]
    mdl = build_model(price)
    sputils.attach_root_node(mdl, mdl.first_stage_costs, [mdl.develop_land, mdl.build_now])
    mdl._mpisppy_probability = 0.5
    return mdl

### Solving model with using Extensive Form

In [8]:
options = {"solver": "scip"}
scenario_names = list(scenarios.keys())
ef = ExtensiveForm(options, scenario_names, scenario_creator)
results = ef.solve_extensive_form()

[   10.59] Initializing SPBase


In [9]:
objective = ef.get_objective_value()
root_solution = ef.get_root_solution()

print(objective)
print(root_solution)

195.0
{'develop_land[1]': 0.0, 'develop_land[2]': 1.0, 'build_now[1]': 0.0, 'build_now[2]': 0.0}


In [10]:
if ef.tree_solution_available:
    ef.write_tree_solution("solutions_tree")

### Solving scenarios one by one

In [11]:
mdl = scenario_creator("s1")
solver = pyo.SolverFactory('scip')
results = solver.solve(mdl)

In [12]:
objective = mdl.objective()
build_now = mdl.build_now.extract_values()
build_later = mdl.build_later.extract_values()
develop_land = mdl.develop_land.extract_values()

print("Build now: ", build_now)
print("Build later: ", build_later)
print("Develop land: ", develop_land)
print("Objective value: ", objective)

Build now:  {1: 1.0, 2: 1.0}
Build later:  {1: -0.0, 2: -0.0}
Develop land:  {1: 1.0, 2: 1.0}
Objective value:  1000.0
